In [1]:
import numpy as np
import pandas as pd
from pyntcloud import PyntCloud as pc
from tqdm import tqdm_notebook as tqdm

from Kinect import Kinect
from Planes import find_plane, distance_from_plane

In [2]:
k = Kinect(debug=True)
k.scale_depth(True)
k.scale_rgb(False)
k.start()
point_cloud = k.get_pointcloud()
#point_cloud[:,3:] *= 255.0
#depth_scale = ((np.abs(np.mean(point_cloud[:,0])) + np.abs(np.mean(point_cloud[:,1]))) / 2) / np.mean(point_cloud[:,2])
#print("Depth scale: {}".format(depth_scale))
#point_cloud[:,2] *= depth_scale
k.stop()

OniDeviceInfo(uri = b'freenect2://0?serial=124535140947', vendor = b'Microsoft', name = b'Kinect', usbVendorId = 31312, usbProductId = 11450)
OniVideoMode(pixelFormat = OniPixelFormat.ONI_PIXEL_FORMAT_DEPTH_1_MM, resolutionX = 512, resolutionY = 424, fps = 30)
OniVideoMode(pixelFormat = OniPixelFormat.ONI_PIXEL_FORMAT_DEPTH_1_MM, resolutionX = 640, resolutionY = 480, fps = 30)
Min depth value: 0
Max depth value: 10000
OniVideoMode(pixelFormat = OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX = 512, resolutionY = 424, fps = 30)
OniVideoMode(pixelFormat = OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX = 1920, resolutionY = 1080, fps = 30)
(217088,)
Depth min: 0.0
Depth max: 4495
Depth scale: 1547.909747901962
Pointcloud calculation time: 0.3782196044921875 secs


In [3]:
points = pd.DataFrame(point_cloud, columns=['x', 'y', 'z', 'red', 'green', 'blue'])
cloud = pc(points)
cloud.plot(IFrame_shape=(1200, 700))

In [4]:
points.describe()

,x,y,z,red,green,blue
count,69048.000000,69048.000000,69048.000000,69048.000000,69048.000000,69048.000000
mean,346.810118,-259.564897,-303.187507,125.556120,118.380489,91.508458
std,129.363257,77.314849,139.680834,45.727003,47.430159,55.307490
min,1.000000,-416.000000,-695.785432,0.000000,0.000000,0.000000
25%,284.000000,-330.000000,-357.412361,108.000000,96.000000,57.000000
50%,381.000000,-252.000000,-211.754054,125.000000,108.000000,70.000000
75%,443.000000,-196.000000,-206.645951,160.000000,154.000000,133.000000
max,510.000000,0.000000,-77.395487,255.000000,255.000000,255.000000


In [5]:
# from: https://stackoverflow.com/a/9271260/6588972
def multidim_intersect(arr1, arr2):
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = np.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])

In [6]:
def multidim_isin_mask(arr1, arr2):
    arr1_view = arr1.view([('', arr1.dtype)] * arr1.shape[1])
    arr2_view = arr2.view([('', arr2.dtype)] * arr2.shape[1])
    isin = np.isin(arr1_view, arr2_view)
    #print(isin)
    #return isin.view(arr1.dtype).reshape(-1, arr1.shape[1])
    return isin

In [7]:
def get_inliers(p1, p2, p3, points, limit):
    p_a, p_b, p_c, p_d = find_plane(p1, p2, p3)
    
    dists = distance_from_plane(points, p_a, p_b, p_c, p_d)
    if dists is False:
        return False
    
    inliers = points[dists<limit]
    
    res = {"Plane": np.array([p_a, p_b, p_c, p_d]),
           "num_inliers": inliers.shape[0],
           "inliers": inliers,
           "dists": dists}

    return res

In [8]:
def remove_points_from_cloud(cloud, points):
    t = multidim_isin_mask(cloud, points)
    tt = np.repeat(t, cloud.shape[1], axis=1)
    c = cloud[~tt].reshape((-1, cloud.shape[1]))
    return c

In [9]:
def angle_between_planes(norm1, norm2):
    return np.arccos(np.dot(norm1, norm2) / (np.linalg.norm(norm1) * np.linalg.norm(norm2)))

In [10]:
def remove_points_by_distance(cloud, fits):
    pass

In [11]:
def get_rand_points(points):
    
    r = np.random.choice(range(len(points)), 3)
    p = points[r]

    return p

In [12]:
a = [[1, 2, 3],
     [4, 5, 6],
     [1, 2, 4],
     [2, 2, 2],
     [3, 3, 3]]

b = [[4, 4, 4],
     [5, 5, 5]]

np.append(a, b, axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 4],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5]])

In [13]:
def do_ransac(pc, iterations, limit=0.001, min_inliers=500):
    best_fit = []
    
    ps = pc[:,:3]
    ps_select = pc[:,:3]
    
    for ite in tqdm(range(iterations)):
        if(ps_select.shape[0] < 2000):
            break
        #print(ps_select.shape)
        p1, p2, p3 = get_rand_points(ps_select)
        #p2 = get_rand_point(ps_select)
        #p3 = get_rand_point(ps_select)
    
        fit = get_inliers(p1, p2, p3, ps, limit)
        if fit is False:
            continue
        #all_inliers = np.append(all_inliers, fit["inliers"][:,:3], axis=0)
        #print(all_inliers.shape)
        
        if fit["num_inliers"] > min_inliers:
            best_fit.append(fit)
            ps_select = remove_points_from_cloud(np.ascontiguousarray(ps_select), np.ascontiguousarray(fit["inliers"]))
        
        
    return best_fit

In [14]:
lim = 2
ran = do_ransac(point_cloud, 300, lim, 5000)

In [15]:
ran = sorted(ran, key=lambda k: k['num_inliers'], reverse=True) 
for r in ran:
    print("Number of inliers: {}".format(r["num_inliers"]))

Number of inliers: 26686
Number of inliers: 8227
Number of inliers: 6412
Number of inliers: 6231
Number of inliers: 5464
Number of inliers: 5227


In [16]:
"""
# merge all similar into largest plane
p_large_norm = np.array(ran[-1]["Plane"][0:3])
print(p_large_norm)
print("")

index_merge = []

for c, r in enumerate(ran):
    if c == 0:
        continue
    norm = r["Plane"][0:3]
    angle = angle_between_planes(p_large_norm, norm)
    print(angle)
    if angle < 3:
        index_merge.append(c)

print(index_merge)

for i in index_merge:
    ran[0]["Plane"] = (np.array(ran[0]["Plane"][0:3]) + np.array(ran[i]["Plane"][0:3])) / 2
    ran[0]["num_inliers"] = ran[0]["num_inliers"] + ran[i]["num_inliers"]
    ran[0]["inliers"] = np.append(ran[0]["inliers"], ran[i]["inliers"])
    ran[0]["dists"] = (ran[0]["dists"] + ran[i]["dists"]) / 2

#for i in index_merge:
#    del ran[i]
"""

'\n# merge all similar into largest plane\np_large_norm = np.array(ran[-1]["Plane"][0:3])\nprint(p_large_norm)\nprint("")\n\nindex_merge = []\n\nfor c, r in enumerate(ran):\n    if c == 0:\n        continue\n    norm = r["Plane"][0:3]\n    angle = angle_between_planes(p_large_norm, norm)\n    print(angle)\n    if angle < 3:\n        index_merge.append(c)\n\nprint(index_merge)\n\nfor i in index_merge:\n    ran[0]["Plane"] = (np.array(ran[0]["Plane"][0:3]) + np.array(ran[i]["Plane"][0:3])) / 2\n    ran[0]["num_inliers"] = ran[0]["num_inliers"] + ran[i]["num_inliers"]\n    ran[0]["inliers"] = np.append(ran[0]["inliers"], ran[i]["inliers"])\n    ran[0]["dists"] = (ran[0]["dists"] + ran[i]["dists"]) / 2\n\n#for i in index_merge:\n#    del ran[i]\n'

In [17]:
pc_show = np.copy(point_cloud)

if len(ran) > 0:
    cond_blue = np.less(ran[0]["dists"], lim)
    np.putmask(pc_show[:,5], cond_blue, 255.0)


if len(ran) > 1:
    cond_green = np.less(ran[1]["dists"], lim)
    np.putmask(pc_show[:,4], cond_green, 255.0)

if len(ran) > 2:
    cond_red = np.less(ran[2]["dists"], lim)
    #np.putmask(pc_show[:,3], cond_red, 255.0)

In [18]:
points = pd.DataFrame(pc_show, columns=['x', 'y', 'z', 'red', 'green', 'blue'])
#points.describe()
cloud = pc(points)
cloud.plot(IFrame_shape=(1200, 700))